In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FirstSparkSessionApp").getOrCreate()

24/12/09 10:21:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.format('json').load('data/2015-summary.json', inferSchema=True, header=True )

In [3]:
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [4]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



# ROW클래스, 단일 레코드(행)을 나타내는 객체 

In [5]:
df.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [6]:
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [7]:
#spark DataTable
df.select("DEST_COUNTRY_NAME").show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
+-----------------+
only showing top 5 rows



In [8]:
df.createOrReplaceTempView("mobility_data")

In [9]:
df.count()

256

In [10]:
df_dup2= df.select('DEST_COUNTRY_NAME').dropDuplicates()
df_dup2.show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|         Anguilla|
|           Russia|
|         Paraguay|
|          Senegal|
|           Sweden|
+-----------------+
only showing top 5 rows



In [11]:
df_dup3= df.select('DEST_COUNTRY_NAME').dropDuplicates().cache() #중복제거 #cache- 메모리에 올리기 
df_dup3.show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows



In [12]:
df_dup.count()

NameError: name 'df_dup' is not defined

In [13]:
df_dup.distinct().count()

NameError: name 'df_dup' is not defined

In [14]:
df.sort('DEST_COUNTRY_NAME').show() #셔플 

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|             Algeria|      United States|    4|
|              Angola|      United States|   15|
|            Anguilla|      United States|   41|
| Antigua and Barbuda|      United States|  126|
|           Argentina|      United States|  180|
|               Aruba|      United States|  346|
|           Australia|      United States|  329|
|             Austria|      United States|   62|
|          Azerbaijan|      United States|   21|
|             Bahrain|      United States|   19|
|            Barbados|      United States|  154|
|             Belgium|      United States|  259|
|              Belize|      United States|  188|
|             Bermuda|      United States|  183|
|             Bolivia|      United States|   30|
|Bonaire, Sint Eus...|      United States|   58|
|              Brazil|      United States|  853|
|British Virgin Is..

In [15]:
from pyspark.sql.functions import expr

In [16]:
df3= df.withColumn('withInCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME')) #컬럼 추가  expr

In [17]:
df3.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withInCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [18]:
#SQL 구문 CASE WHEN > 수치형 변수 > 명목형 변수로 변환 >파생변수 
df4= df.withColumn('category', expr('CASE WHEN count<10 THEN "under" WHEN count>=10 THEN "upper" END'))

In [19]:
df4.show(5)

+-----------------+-------------------+-----+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|
+-----------------+-------------------+-----+--------+
|    United States|            Romania|   15|   upper|
|    United States|            Croatia|    1|   under|
|    United States|            Ireland|  344|   upper|
|            Egypt|      United States|   15|   upper|
|    United States|              India|   62|   upper|
+-----------------+-------------------+-----+--------+
only showing top 5 rows



In [20]:
#count 2배 

In [21]:
df5 = df4.withColumn('withInCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME'))

In [22]:
df5.show()

+--------------------+-------------------+-----+--------+-------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|withInCountry|
+--------------------+-------------------+-----+--------+-------------+
|       United States|            Romania|   15|   upper|        false|
|       United States|            Croatia|    1|   under|        false|
|       United States|            Ireland|  344|   upper|        false|
|               Egypt|      United States|   15|   upper|        false|
|       United States|              India|   62|   upper|        false|
|       United States|          Singapore|    1|   under|        false|
|       United States|            Grenada|   62|   upper|        false|
|          Costa Rica|      United States|  588|   upper|        false|
|             Senegal|      United States|   40|   upper|        false|
|             Moldova|      United States|    1|   under|        false|
|       United States|       Sint Maarten|  325|   upper|       

In [23]:
#groupby 집계  

In [24]:
df5.groupBy("withInCountry").count().show()

+-------------+-----+
|withInCountry|count|
+-------------+-----+
|         true|    1|
|        false|  255|
+-------------+-----+



In [25]:
df5.groupBy("category").count().show()

+--------+-----+
|category|count|
+--------+-----+
|   under|   48|
|   upper|  208|
+--------+-----+



# Projection 과 Filter
### select a,b,c from TableA  # projection > column > Transformation select('colname')
### where a>10  #filter > Row >Transformation where('cond')

In [26]:
df6 = df5.where('count<5' ) #행 꺼내기 

In [27]:
df6.show()

+--------------------+-------------------+-----+--------+-------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|withInCountry|
+--------------------+-------------------+-----+--------+-------------+
|       United States|            Croatia|    1|   under|        false|
|       United States|          Singapore|    1|   under|        false|
|             Moldova|      United States|    1|   under|        false|
|               Malta|      United States|    1|   under|        false|
|             Algeria|      United States|    4|   under|        false|
|       United States|          Gibraltar|    1|   under|        false|
|Saint Vincent and...|      United States|    1|   under|        false|
|            Suriname|      United States|    1|   under|        false|
|       United States|             Cyprus|    1|   under|        false|
|       United States|           Malaysia|    3|   under|        false|
|            Thailand|      United States|    3|   under|       

In [45]:
df6.count()

46

In [ ]:
where a>10 and b!=1

In [28]:
df7 = df5.where('count<5' ).where('ORIGIN_COUNTRY_NAME != "United States"')
df7.show()

+-----------------+-------------------+-----+--------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|withInCountry|
+-----------------+-------------------+-----+--------+-------------+
|    United States|            Croatia|    1|   under|        false|
|    United States|          Singapore|    1|   under|        false|
|    United States|          Gibraltar|    1|   under|        false|
|    United States|             Cyprus|    1|   under|        false|
|    United States|           Malaysia|    3|   under|        false|
|    United States|            Vietnam|    2|   under|        false|
|    United States|            Estonia|    1|   under|        false|
|    United States|            Hungary|    3|   under|        false|
|    United States|           Thailand|    4|   under|        false|
|    United States|            Liberia|    2|   under|        false|
|    United States|              Malta|    2|   under|        false|
|    United States|          Lithu

In [29]:
#프로젝션, 필터링 연습

In [ ]:
df5.groupBy('withInCountry').count().show()

In [ ]:
#국내 여행 카운트를 확인합니다.
df5.where('withInCountry==true').show()

In [ ]:
#국내 여행이 아니면서 가장 횟수가 많은 ORIGIN_COUNTRY_NAME top 10을  추출해보세요

In [ ]:
#국내 여행이 아니면서 가장 횟수가 적은 ORIGIN_COUNTRY_NAME top 10을  추출해보세요

In [ ]:
#도착국가별 count 총합이 가장 많은 top10을 추출해 보세요

In [35]:
spark.stop()

# 집계함수
ㅡ

In [36]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SecondSparkSessionApp").getOrCreate()

In [37]:
df= spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/emp.csv')

In [38]:
df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: string (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [39]:
df.select('ename', 'deptno').show()

+------+------+
| ename|deptno|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [40]:
df.select('ename', 'deptno').where('deptno=20').show() #선택 추출 

+-----+------+
|ename|deptno|
+-----+------+
|SMITH|    20|
|JONES|    20|
|SCOTT|    20|
|ADAMS|    20|
| FORD|    20|
+-----+------+



In [ ]:
#카운트 집계 

In [42]:
from pyspark.sql.functions import count

In [43]:
df.select(count('job')).show() #job 몇번 들어있는지 #null값 제외 

+----------+
|count(job)|
+----------+
|        15|
+----------+



In [44]:
df.selectExpr('count(*)').show() #안에 들어가는게 표현식이다.  ,null값 포함 , 위와 차이로 null값 유무 파악 

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [45]:
df.select('job').distinct().show()

+---------+
|      job|
+---------+
|  ANALYST|
| SALESMAN|
|    CLERK|
|  MANAGER|
|PRESIDENT|
+---------+



In [46]:
df.select('job').distinct().count() #정확 

5

In [53]:
from pyspark.sql.functions import countDistinct, approx_count_distinct

In [54]:
df.select(countDistinct('job')).show() #성능 더 좋음 , 꺼낼 때 바로 시행 , #근사치 

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [56]:
df.select(approx_count_distinct('job', 0.1)).show()  #오차범위 

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [57]:
from pyspark.sql.functions import first,last #API다 외울 수는 없음. 코드를 작성해서 필요한 기능 찾기 

In [58]:
df. select(first('ename'), last('ename')).show()

+------------+-----------+
|first(ename)|last(ename)|
+------------+-----------+
|       SMITH|       JACK|
+------------+-----------+



In [ ]:
#min, max

In [69]:
from pyspark.sql.functions import max, min , sum, avg

In [61]:
df.select(max('ename'), min('ename')).show() 

+----------+----------+
|max(ename)|min(ename)|
+----------+----------+
|      WARD|     ADAMS|
+----------+----------+



In [64]:
df.select(count('empno'),count('*'), max('ename'), min('ename')).show()

+------------+--------+----------+----------+
|count(empno)|count(1)|max(ename)|min(ename)|
+------------+--------+----------+----------+
|          15|      15|      WARD|     ADAMS|
+------------+--------+----------+----------+



In [67]:
df.select(sum('sal')).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [68]:
#sal 컬럼값의 중복을 제거하고 합산 
#df.select('sal').distinct().sum()
df.selectExpr('sum( distinct sal)').show() #distinct를 해서 가져옴 ,sum( distinct saㅣ- 이건 표현식이기에 selectExpr 사용 

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [71]:
#alias
dfs= df.select(count('sal').alias('total_tx') , 
                   sum('sal').alias('total_salary'),
                   #avg('sal').alias('avg_salary')
                   expr('mean(sal)').alias('mean_salary')
              )
dfs.show()

+--------+------------+------------------+
|total_tx|total_salary|       mean_salary|
+--------+------------+------------------+
|      15|       32225|2148.3333333333335|
+--------+------------+------------------+



In [83]:
from pyspark.sql.functions import round

#round (data, 자릿수)- 해보기 
dfs= df.select(count('sal').alias('total_tx') , 
                   sum('sal').alias('total_salary'),
                   #avg('sal').alias('avg_salary')
                   round(expr('mean(sal)'),2).alias('mean_salary')
              )


In [84]:
dfs.show()

+--------+------------+-----------+
|total_tx|total_salary|mean_salary|
+--------+------------+-----------+
|      15|       32225|    2148.33|
+--------+------------+-----------+



# 그룹화

In [73]:
df.groupBy('job').count().show() #집계 후 그룹화 해줘야 됨. 

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [75]:
#agg() 집계함수 적용 
dfs= df.groupBy('job').agg( expr('avg(sal) as_SAL_AVG'))

In [76]:
dfs.show()

+---------+------------------+
|      job|        as_SAL_AVG|
+---------+------------------+
|  ANALYST|            3000.0|
| SALESMAN|            1400.0|
|    CLERK|            1470.0|
|  MANAGER|2758.3333333333335|
|PRESIDENT|            5000.0|
+---------+------------------+



In [77]:
#표준펴ㄴ차 

#1. sql=function - stddev()
dfs= df.groupBy('job').agg(expr('stddev_pop(sal) as SAL_STDEV')).show()
#2. sql expression

+---------+------------------+
|      job|         SAL_STDEV|
+---------+------------------+
|  ANALYST|               0.0|
| SALESMAN|154.11035007422439|
|    CLERK| 880.6815542521599|
|  MANAGER|223.91714737574006|
|PRESIDENT|               0.0|
+---------+------------------+



In [ ]:
#round() 처리

In [85]:
dfs= df.groupBy('job').agg(round(expr('stddev_pop(sal) as SAL_STDEV'),2)).show()

+---------+----------------------------------------+
|      job|round(stddev_pop(sal) AS `SAL_STDEV`, 2)|
+---------+----------------------------------------+
|  ANALYST|                                     0.0|
| SALESMAN|                                  154.11|
|    CLERK|                                  880.68|
|  MANAGER|                                  223.92|
|PRESIDENT|                                     0.0|
+---------+----------------------------------------+



In [ ]:
#groupBy집계 

# 윈도우 함수 

In [ ]:
순위, 정렬, - rank, row_number , dense_rank
누계 - sum, avg, max, min + over()
이동평균, 이동합계 - over+ rowsBetween, rangeBetween
시차, 서행 -leg, lead 

ex) 세션 구간내 분석, 특정시간 동안 일어난 활동 그룹화 


In [ ]:
1. partitionBy() > 소그룹으로 나눈다.
2. orderBy() > 소그룹 내 정렬
3. rowBetween() , rangeBetween()
4. over()

In [88]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc,rank

#윈도우 명세 설정
windowspec=Window.orderBy(desc('sal'))  # 범위 정함 

In [89]:
#순위객체 
salAllRank= rank().over(windowspec) #rank 데이터 생성 

In [90]:
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [91]:
#데이터프레임에 컬럼으로 추가 > 액션 
df.withColumn("salary_rank",salAllRank).show()

24/12/09 13:39:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|null|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|         10|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|         11|
| 7654|MARTIN| SALES

In [101]:
from pyspark.sql.functions import col
df.select( 'empno', salAllRank.alias('salary_rank') ).show()  

ImportError: cannot import name 'col' from 'pyspark.sql.window' (/opt/spark/python/pyspark/sql/window.py)

In [98]:
#windowspec=Window.orderBy(desc('sal'))
windowspec1= Window.partitionBy('job').orderBy(desc('sal'))

In [102]:
salJobRank= rank().over(windowspec)

In [104]:
df.select(
    'job', 'ename', 'sal',
    salJobRank.alias('salJobRank')
).show()

+---------+------+----+----------+
|      job| ename| sal|salJobRank|
+---------+------+----+----------+
|  ANALYST| SCOTT|3000|         1|
|  ANALYST|  FORD|3000|         1|
| SALESMAN| ALLEN|1600|         1|
| SALESMAN|TURNER|1500|         2|
| SALESMAN|  WARD|1250|         3|
| SALESMAN|MARTIN|1250|         3|
|    CLERK|  JACK|3200|         1|
|    CLERK|MILLER|1300|         2|
|    CLERK| ADAMS|1100|         3|
|    CLERK| JAMES| 950|         4|
|    CLERK| SMITH| 800|         5|
|  MANAGER| JONES|2975|         1|
|  MANAGER| BLAKE|2850|         2|
|  MANAGER| CLARK|2450|         3|
|PRESIDENT|  KING|5000|         1|
+---------+------+----+----------+



In [107]:
# 부서별 급여 순위 생성  - 강사님 참고하기 

dept_window_spec = Window.partitionBy('deptno').orderBy(desc('sal'))

In [ ]:
new_df = df.withColumn( 'dept_salary_rank', rank().over(dept_window_spec))
new_df.select(
    'ename', 'deptno', 'sal', 'dept_salary_rank'
).show()

In [ ]:
from pyspark.sql.functions import row_number
new_df = df.withColumn( 'dept_salary_rank', row_number().over(dept_window_spec))
new_df.select(
    'ename', 'deptno', 'sal', 'dept_salary_rank'
).show()

In [ ]:
from pyspark.sql.functions import dense_rank
new_df = df.withColumn( 'dept_salary_rank', dense_rank().over(dept_window_spec))
new_df.select(
    'ename', 'deptno', 'sal', 'dept_salary_rank'
).show()

In [ ]:
#누적 급여 계산 

In [112]:
from pyspark.sql.functions import dense_rank
sum_window_spec= Window.partitionBy('deptno').orderBy('empno')


In [114]:
new_df= df.withColumn('cum_salary',sum('sal').over(sum_window_spec))
new_df.select(
    'ename', 'deptno', 'sal', 'cum_salary'
).show()

+------+------+----+----------+
| ename|deptno| sal|cum_salary|
+------+------+----+----------+
| SMITH|    20| 800|       800|
| JONES|    20|2975|      3775|
| SCOTT|    20|3000|      6775|
| ADAMS|    20|1100|      7875|
|  FORD|    20|3000|     10875|
| CLARK|    10|2450|      2450|
|  KING|    10|5000|      7450|
|MILLER|    10|1300|      8750|
|  JACK|    70|3200|      3200|
| ALLEN|    30|1600|      1600|
|  WARD|    30|1250|      2850|
|MARTIN|    30|1250|      4100|
| BLAKE|    30|2850|      6950|
|TURNER|    30|1500|      8450|
| JAMES|    30| 950|      9400|
+------+------+----+----------+



In [ ]:
#부서별 평균 급여 avg('sal')

#sql select 문 

SELECT #projsction
ename, deptno, sal,
avg('sal').over(partition by deptno) as dept_avg_salary
from emp; #filter 

In [ ]:
from pyspark.sql.functions import dense_rank
avg_window_spec = Window.partitionBy('deptno')
new_avg_df = df.withColumn('dept_avg_salary', avg('sal').over(avg_window_spec))
new_avg_df.select('empno', 'deptno', 'dept_avg_salary',).show()

In [115]:
windowspec= Window.partitionBy('deptno')

In [119]:
df_with_avg_sal= df.withColumn('avg_sal',round(avg('sal').over(windowspec),2))

In [121]:
df_with_avg_sal.select('deptno', 'sal', 'avg_sal').show() #F import 해보기 / groupBy 해보기 

+------+----+-------+
|deptno| sal|avg_sal|
+------+----+-------+
|    20| 800| 2175.0|
|    20|2975| 2175.0|
|    20|3000| 2175.0|
|    20|1100| 2175.0|
|    20|3000| 2175.0|
|    10|2450|2916.67|
|    10|5000|2916.67|
|    10|1300|2916.67|
|    70|3200| 3200.0|
|    30|1600|1566.67|
|    30|1250|1566.67|
|    30|1250|1566.67|
|    30|2850|1566.67|
|    30|1500|1566.67|
|    30| 950|1566.67|
+------+----+-------+



In [125]:
#강사님
from pyspark.sql.functions import dense_rank
avg_window_spec = Window.partitionBy('deptno')
new_avg_df= df.withColumn('dept_avg_salary', avg('sal').over(avg_window_spec))
new_avg_df.select('deptno', 'dept_avg_salary').show()

+------+------------------+
|deptno|   dept_avg_salary|
+------+------------------+
|    20|            2175.0|
|    20|            2175.0|
|    20|            2175.0|
|    20|            2175.0|
|    20|            2175.0|
|    10|2916.6666666666665|
|    10|2916.6666666666665|
|    10|2916.6666666666665|
|    70|            3200.0|
|    30|1566.6666666666667|
|    30|1566.6666666666667|
|    30|1566.6666666666667|
|    30|1566.6666666666667|
|    30|1566.6666666666667|
|    30|1566.6666666666667|
+------+------------------+



In [ ]:
#sql select 문으로 바꿔보기 - 연습 많이 해보기 
select 
ename, deptno, sal
LAG(sal) OVER()

In [ ]:
#Lead, Lag 이전 급여, 이후 급여 

In [ ]:
#DF
from pyspark.sql.import functions as F

row_window_spec = Window.partitionBy('deptno').orderBy('empno')

#이전 급여
lead_lagg_df= df.withColumn('prev_salary', lag('sal').over(row_window_spec))\
            .withColumn('next_salary', lag('sal').over(row_window_spec))
 
new_avg_df.select('ename' ,'deptno','sal', 'prev_salary',  'dept_avg_salary').show()   # 다시 봐보기 

# rollup, cube

over(), groupBy()
rollup : 계층적집계, 부분합(subtotal), 총합(grandtotal) (대부류,중부류)
cube : 모든 값으로 부분합 , 결합가능한 모든 값의 부분합을 구한다. 

In [1]:
![image.png](attachment:c3d0c6ad-29c8-4d6c-81c3-360a6ac841ad.png)

/bin/bash: -c: line 0: syntax error near unexpected token `attachment:c3d0c6ad-29c8-4d6c-81c3-360a6ac841ad.png'
/bin/bash: -c: line 0: `[image.png](attachment:c3d0c6ad-29c8-4d6c-81c3-360a6ac841ad.png)'


In [126]:
df

DataFrame[empno: int, ename: string, job: string, mgr: int, hiredate: string, sal: int, comm: int, deptno: int]

In [132]:
#그룹화 >소계 

df.groupBy('deptno', 'job').agg(count('*'),sum('sal')).orderBy('deptno', 'job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [133]:
df.rollup('deptno', 'job').agg(count('*'),sum('sal')).orderBy('deptno', 'job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  null|     null|      15|   32225|
|    10|     null|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     null|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     null|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     null|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [134]:
df.cube('deptno', 'job').agg(count('*'),sum('sal')).orderBy('deptno', 'job').show() 

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  null|     null|      15|   32225|
|  null|  ANALYST|       2|    6000|
|  null|    CLERK|       5|    7350|
|  null|  MANAGER|       3|    8275|
|  null|PRESIDENT|       1|    5000|
|  null| SALESMAN|       4|    5600|
|    10|     null|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     null|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     null|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     null|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [ ]:
# 롤업 - > 대 중분류 최대 / 최솟값

#sql 버전 
select
    deptno, job,
    max(sal) as max_sal,
    min(sal) as min_sal,
from emp
group by rollup(deptno, job)
order by deptno, job ;


In [135]:
# 롤업 - > 대 중분류 최대 / 최솟값

de_rollup= df.rollup('deptno', 'job').agg(
    count('*').alias('count'),
    sum('sal').alias('total_salary'),
    max('sal').alias('max_salary'),
    min('sal').alias('min_salary')
).orderBy('deptno', 'job')

de_rollup.show()

+------+---------+-----+------------+----------+----------+
|deptno|      job|count|total_salary|max_salary|min_salary|
+------+---------+-----+------------+----------+----------+
|  null|     null|   15|       32225|      5000|       800|
|    10|     null|    3|        8750|      5000|      1300|
|    10|    CLERK|    1|        1300|      1300|      1300|
|    10|  MANAGER|    1|        2450|      2450|      2450|
|    10|PRESIDENT|    1|        5000|      5000|      5000|
|    20|     null|    5|       10875|      3000|       800|
|    20|  ANALYST|    2|        6000|      3000|      3000|
|    20|    CLERK|    2|        1900|      1100|       800|
|    20|  MANAGER|    1|        2975|      2975|      2975|
|    30|     null|    6|        9400|      2850|       950|
|    30|    CLERK|    1|         950|       950|       950|
|    30|  MANAGER|    1|        2850|      2850|      2850|
|    30| SALESMAN|    4|        5600|      1600|      1250|
|    70|     null|    1|        3200|   

In [137]:
# 강사님
df.rollup('deptno', 'job').agg(max('sal'), min('sal')).orderBy('deptno', 'job').show()

+------+---------+--------+--------+
|deptno|      job|max(sal)|min(sal)|
+------+---------+--------+--------+
|  null|     null|    5000|     800|
|    10|     null|    5000|    1300|
|    10|    CLERK|    1300|    1300|
|    10|  MANAGER|    2450|    2450|
|    10|PRESIDENT|    5000|    5000|
|    20|     null|    3000|     800|
|    20|  ANALYST|    3000|    3000|
|    20|    CLERK|    1100|     800|
|    20|  MANAGER|    2975|    2975|
|    30|     null|    2850|     950|
|    30|    CLERK|     950|     950|
|    30|  MANAGER|    2850|    2850|
|    30| SALESMAN|    1600|    1250|
|    70|     null|    3200|    3200|
|    70|    CLERK|    3200|    3200|
+------+---------+--------+--------+



In [ ]:
#select 로도  

In [138]:
#job 별 평균급여 job, avg_salary, total_salary 
df_job_salary = df.groupBy('job').agg(
    round(avg('sal'),2).alias('avg_salary'),
    sum('sal').alias('total_salary'))

In [139]:
df_job_salary.show()

+---------+----------+------------+
|      job|avg_salary|total_salary|
+---------+----------+------------+
|  ANALYST|    3000.0|        6000|
| SALESMAN|    1400.0|        5600|
|    CLERK|    1470.0|        7350|
|  MANAGER|   2758.33|        8275|
|PRESIDENT|    5000.0|        5000|
+---------+----------+------------+



In [ ]:
#deptno, job 평균급여, 최대급여를 모두 소계를 냅니다. cube 

In [142]:
df_cube= df.cube('deptno', 'job').agg(count('*'),
     round(avg('sal'),2).alias('avg_salary'),
     max('sal').alias('max_salary')
          ).orderBy('deptno', 'job')

df_cube.show()   #더 봐보기 

+------+---------+--------+----------+----------+
|deptno|      job|count(1)|avg_salary|max_salary|
+------+---------+--------+----------+----------+
|  null|     null|      15|   2148.33|      5000|
|  null|  ANALYST|       2|    3000.0|      3000|
|  null|    CLERK|       5|    1470.0|      3200|
|  null|  MANAGER|       3|   2758.33|      2975|
|  null|PRESIDENT|       1|    5000.0|      5000|
|  null| SALESMAN|       4|    1400.0|      1600|
|    10|     null|       3|   2916.67|      5000|
|    10|    CLERK|       1|    1300.0|      1300|
|    10|  MANAGER|       1|    2450.0|      2450|
|    10|PRESIDENT|       1|    5000.0|      5000|
|    20|     null|       5|    2175.0|      3000|
|    20|  ANALYST|       2|    3000.0|      3000|
|    20|    CLERK|       2|     950.0|      1100|
|    20|  MANAGER|       1|    2975.0|      2975|
|    30|     null|       6|   1566.67|      2850|
|    30|    CLERK|       1|     950.0|       950|
|    30|  MANAGER|       1|    2850.0|      2850|


In [143]:
spark.stop()